In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [7]:
!kaggle datasets download -d hj5992/bank-churn-modelling

bank-churn-modelling.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!chmod 600 /root/.kaggle/kaggle.json


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from zipfile import ZipFile
zip_file = ZipFile('/content/bank-churn-modelling.zip')


In [0]:
df=pd.read_csv(zip_file.open('Churn_Modelling.csv'))

In [0]:
df.head(5)

In [0]:
df.dtypes

In [0]:
df = df.drop(columns = ["RowNumber", "CustomerId", "Surname"])

In [0]:
df.head(5)

In [16]:
#feature scaling
!pip install category_encoders
import category_encoders as ce
encoder  = ce.BinaryEncoder(cols = ['Geography', 'Gender'])
df_binary = encoder.fit_transform(df)

    100% |████████████████████████████████| 61kB 1.8MB/s 


In [0]:
df_binary.head(5)

In [0]:
y = df_binary.iloc[:, 13].values
x = df_binary.iloc[:, 0:13].values

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.025, random_state = 0)
x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size = 0.025, random_state = 0)

In [21]:
import tensorflow as tf
import tensorflow.keras 

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
#k-fold on dev set

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
acc = []
for train, test in kfold.split(x_dev, y_dev):
  cf = Sequential()
  cf.add(Dense(output_dim = 8, init = 'uniform', activation = 'relu', input_dim = 13))
  cf.add(Dense(output_dim = 4, init = 'uniform', activation = 'relu'))
  cf.add(Dense(output_dim = 8, init = 'uniform', activation = 'relu'))
  cf.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
  cf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  cf.fit(x_dev[train], y_dev[train], batch_size = 10, nb_epoch = 25)
  scores = cf.evaluate(x_dev[test], y_dev[test], verbose=0)
  acc.append(scores[1] * 100)

In [25]:
  cf = Sequential()
  cf.add(Dense(output_dim = 16, init = 'uniform', activation = 'relu', input_dim = 13))
  cf.add(Dense(output_dim = 8, init = 'uniform', activation = 'relu', input_dim = 16))
  cf.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
  cf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  cf.fit(x_train, y_train, batch_size = 10, nb_epoch = 25)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=13, units=16, kernel_initializer="uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=16, units=8, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/25
9506/9506 [==============================] - 5s 524us/step - loss: 0.4593 - acc: 0.7964
Epoch 2/25
9506/9506 [==============================] - 4s 459us/step - loss: 0.4221 - acc: 0.8032
Epoch 3/25
9506/9506 [==============================] - 4s 459us/step - loss: 0.4165 - acc: 0.8268
Epoch 4/25
9506/9506 [==============================] - 4s 457us/step - loss: 0.4138 - acc: 0.8314
Epoch 5/25
9506/9506 [==============================] - 4s 462us/step - loss: 0.4108 - acc: 0.8327
Epoch 6/25
9506/9506 [==============================] - 4s 457us/step - loss: 0.4086 - acc: 0.8336
Epoch 7/25
9506/9506 [==============================] - 4s 459us/step - loss: 0.4051 - acc: 0.8351
Epoch 8/25
9506/9506 [==============================] - 4s 457us/step - loss: 0.4028 - acc: 0.8351
Epoch 9/25
9506/9506 [==============================] - 4s 455us/step - loss: 0.4003 - acc: 0.8355
Epoch 10/25
9506/9506 [==============================] - 4s 455us/step - loss: 0.3992 - acc: 0.8371
Epoch 11/

In [23]:
np.mean(acc)



79.92753744125366

In [24]:
np.std(acc)

0.9762448386120193

In [0]:
y_pred = cf.predict(x_test)
y_pred = (y_pred > 0.5)

In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [29]:
cm

array([[185,  11],
       [ 21,  33]])

In [31]:
(185+33)/(185+33+33)

0.8685258964143426

In [35]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

0.872
